Loading Dataset and Preprocessing

In [2]:
import pandas as pd

data = pd.read_csv('Online Retail.csv')

data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [6]:
columns_to_drop = ['InvoiceNo', 'StockCode','Country']
data = data.drop(columns= columns_to_drop)

data.head()

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850
1,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850
2,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850
4,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850


In [7]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['Quantity'] = data['Quantity'].abs()
data['SaleValue'] = data['Quantity'] * data['UnitPrice']
data = data.set_index('InvoiceDate')

data.head()

,Description,Quantity,UnitPrice,CustomerID,SaleValue
InvoiceDate,,,,,
2010-12-01 08:26:00,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850,15.30
2010-12-01 08:26:00,WHITE METAL LANTERN,6,3.39,17850,20.34
2010-12-01 08:26:00,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850,22.00
2010-12-01 08:26:00,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850,20.34
2010-12-01 08:26:00,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850,20.34


In [8]:
columns_to_drop = ['Quantity', 'UnitPrice']
data = data.drop(columns=columns_to_drop)

data.head()

,Description,CustomerID,SaleValue
InvoiceDate,,,
2010-12-01 08:26:00,WHITE HANGING HEART T-LIGHT HOLDER,17850,15.30
2010-12-01 08:26:00,WHITE METAL LANTERN,17850,20.34
2010-12-01 08:26:00,CREAM CUPID HEARTS COAT HANGER,17850,22.00
2010-12-01 08:26:00,KNITTED UNION FLAG HOT WATER BOTTLE,17850,20.34
2010-12-01 08:26:00,RED WOOLLY HOTTIE WHITE HEART.,17850,20.34


In [10]:
data.isnull().sum()

Description    1454
CustomerID        0
SaleValue         0
dtype: int64

In [12]:
data.dropna(inplace=True)
data.isnull().sum()

Description    0
CustomerID     0
SaleValue      0
dtype: int64